# Houm Data Engineer Challenge  Javier Urrecha

## Introducción


Houm es una startup que permite administrar, arrendar y vender propiedades rápido, seguro y fácil a miles de usuarios en Latinoamérica. En su desafío por entregar la mejor experiencia de usuarios, la empresa y su head de operaciones le pide hacer un estudio sobre el comportamiento de sus visitas en torno al clima. Es por esto que lo contactan a usted para dar respuestas a las preguntas de la compañía.



	Carlos, nuestro data scientist, le informa que existen tres fuentes de información relevantes, las cuales contienen información de la operación que podrían ser de utilidad. Los tres archivos se detallan a continuación (presentes en el archivo dataset.zip adjunto):


<li><b>properties.csv:</b> Contiene información básica de las características de la propiedad y su información geográfica. Las columnas de este archivo son las siguientes: property_id, business_type, type, bedrooms, bathrooms, latitude, longitude, locality, city & country.</li>

<li><b>users.csv:</b> Contiene la información de los propietarios y su relación con la propiedad. Las columnas de este archivo son las siguientes: property_id, user_id, name, last_name & country.</li>

<li><b>visits.csv:</b>
Contiene la información de los clientes que se han registrado en alguna visita a una propiedad. Las columnas de este archivo son las siguientes: schedule_id, property_id, begin_date, end_date, type_visit & status. Este ultimo campo, puede tomar valor según el estado de la visita y permite verificar si las visitas están agendadas y aún no se realizan (scheduled), canceladas (cancelled) o ya realizadas (Done).</li>

	Por otro lado, Carlos le comenta que existe un servicio para obtener información de las condiciones climáticas de cada país. Se puede consultar este servicio por medio de su API, la cual esta documentada aquí. Se puede consultar la temperatura y clima de una localidad mediante la siguiente consulta. 
<b>Ejemplo de Request API Wheather VisualCrossing:</b>

﻿ https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat},{lng}/{start_date}/{end_date}?key={API_KEY}&include=days

<b>Posibles condiciones climáticas retornada por la API:</b>

https://github.com/visualcrossing/WeatherApi/blob/master/lang/en.txt

<b>Nota:</b> Para la utilización de la api se debe proveer el siguiente API KEY. Esta tiene un límite de 1.000 registros al día. Debes registrarte en https://www.visualcrossing.com/sign-up para obtener la KEY.

Preguntas del desafío:

<li>¿Cuántas visitas se realizaron en total?</li>
<li>¿Cuál es el promedio de propiedades por propietario?</li>
<li>¿Cuál era la temperatura promedio de todas las visitas que realizó en la propiedad del propietario con ID 2?</li>
<li>¿Cuál es la temperatura promedio de las visitas para los días con lluvia?</li>
<li>¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?</li>


	Formato de entrega:

Para entregar este desafío te solicitamos compartirnos un link de un repositorio en github o gitlab. Este repositorio debe contener un archivo en formato ipynb o py con las respuestas. Debes entregar un archivo requierements.txt con las versiones de las librerías ocupadas.

In [1]:
# Libraries
import numpy as np
import pandas as pd
import requests
import datetime

## 1. Analisis exploratorio de los datos

In [2]:
# Transformamos los archivos de csv en dataframe
properties = pd.read_csv("properties.csv")
users = pd.read_csv("users.csv")
visits = pd.read_csv("visits.csv")

## 1.1 Propiedades: 

In [25]:
# Obtener el tamaño de los datos
print("Shape: ",properties.shape)

# Vistazo inicial de los datos
properties.head()

Shape:  (80, 15)


,property_id,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
0,1,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.058040,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
1,2,departamento,Rental & Sale,1,1,2,NaN,1,False,4.623068,-74.074030,Teusaquillo,Bogotá,Región De Cundinamarca,Colombia
2,3,departamento,Rental,2,2,2,NaN,0,False,4.723909,-74.042336,Usaquen,Bogotá,Región De Cundinamarca,Colombia
3,4,departamento,Rental & Sale,3,2,1,0.0,0,False,4.721716,-74.057976,Suba,Bogotá,Región De Cundinamarca,Colombia
4,5,departamento,Rental & Sale,2,3,2,0.0,1,False,4.688163,-74.043300,Usaquen,Bogotá,Región De Cundinamarca,Colombia


In [31]:
# Revisamos el typo de los datos para ver si necesitamos hacer transformaciones y ver donde puede haber valores nulos
properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   property_id    80 non-null     int64  
 1   type_house     80 non-null     object 
 2   business_type  80 non-null     object 
 3   bedrooms       80 non-null     int64  
 4   bathrooms      80 non-null     int64  
 5   parking_lots   80 non-null     int64  
 6   services       19 non-null     float64
 7   balcony        80 non-null     int64  
 8   pool           80 non-null     bool   
 9   latitude       80 non-null     float64
 10  longitude      80 non-null     float64
 11  localidad      80 non-null     object 
 12  city           80 non-null     object 
 13  region         80 non-null     object 
 14  country        80 non-null     object 
dtypes: bool(1), float64(3), int64(5), object(6)
memory usage: 9.0+ KB


In [34]:
# Con el método de decribe entendemos si hay outliers y dónde se concentran los datos
properties.describe()

,property_id,bedrooms,bathrooms,parking_lots,services,balcony,latitude,longitude
count,80.0000,80.000000,80.000000,80.000000,19.000000,80.000000,80.000000,80.000000
mean,40.5000,2.275000,1.725000,0.650000,5264.526316,0.137500,4.685079,-74.116246
std,23.2379,0.871126,0.655551,0.713345,22941.242048,0.346547,0.062736,0.062496
min,1.0000,1.000000,1.000000,0.000000,0.000000,0.000000,4.586188,-74.245544
25%,20.7500,2.000000,1.000000,0.000000,0.000000,0.000000,4.644076,-74.158101
50%,40.5000,2.000000,2.000000,1.000000,1.000000,0.000000,4.685236,-74.111243
75%,60.2500,3.000000,2.000000,1.000000,3.000000,0.000000,4.722041,-74.057694
max,80.0000,5.000000,4.000000,2.000000,100000.000000,1.000000,4.926684,-74.023470


<b>Notas:</b>
<li>Los datos de servicios destacan inmediatamente donde el máximo es de 100000 pero el promedio esta cerca de 0, es interesante por que es la columna
con más datos nulos (61). Para este ejercicio no nos preocuparemos por esta columna pero deberíamos entender más acerca de por qué sus datos no son íntegros.</li>
<li>Son pocos datos por lo que podemos inferir que los resultados numéricos no deberían ser muy elevados</li>
<li>Tenemos 80 propiedades con 15 características.</li>

## 1.2 Usuarios

Analizaremos la tabla de usuarios para comprobar que coincide con los datos de propiedades repitiendo el proceso de analisis.

In [49]:
# Obtener el tamaño de los datos
print("Shape: ",users.shape)

# Vistazo inicial de los datos
users.head()

Shape:  (80, 5)


,property_id,user_id,first_name,last_name,address
0,31,1,Josephine,Darakjy,4 B Blue Ridge Blvd
1,34,2,Art,Venere,8 W Cerritos Ave #54
2,48,3,Lenna,Paprocki,639 Main St
3,27,4,Donette,Foller,34 Center St
4,40,5,Simona,Morasca,3 Mcauley Dr


In [43]:
# Revisamos el typo de los datos para ver si necesitamos hacer transformaciones y ver donde puede haber valores nulos
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  80 non-null     int64 
 1   user_id      80 non-null     int64 
 2   first_name   80 non-null     object
 3   last_name    80 non-null     object
 4   address      80 non-null     object
dtypes: int64(2), object(3)
memory usage: 3.2+ KB


In [45]:
# Con el método de decribe entendemos si hay outliers y dónde se concentran los datos
users.describe()

,property_id,user_id
count,80.0000,80.0000
mean,40.5000,40.5000
std,23.2379,23.2379
min,1.0000,1.0000
25%,20.7500,20.7500
50%,40.5000,40.5000
75%,60.2500,60.2500
max,80.0000,80.0000


In [47]:
# Checaremos si hay más propiedades por usuario
list(users.property_id.value_counts())

31    1
34    1
56    1
55    1
54    1
     ..
8     1
7     1
6     1
4     1
80    1
Name: property_id, Length: 80, dtype: int64

<b>Notas:</b>
<li>La longitud de los datos coincide con las de las propiedades, y cada una de estas pertenece a un usuario</li>
<li>Los datos están limpios y no resaltan outliers.</li>

## 1.3 Visitas

Repetimos el proceso con las visitas

In [50]:
# Obtener el tamaño de los datos
print("Shape: ",visits.shape)

# Vistazo inicial de los datos
visits.head()

Shape:  (425, 6)


,scheduled_id,property_id,begin_date,end_date,type_visit,status
0,169548,1,2022-01-13T10:00:00-03:00,2022-01-13T12:00:00-03:00,Visit,Cancelled
1,184763,1,2022-01-26T18:00:00-03:00,2022-01-26T20:00:00-03:00,Visit,Cancelled
2,186092,1,2022-01-28T12:00:00-03:00,2022-01-28T14:00:00-03:00,Visit,Cancelled
3,182497,1,2022-01-23T11:00:00-03:00,2022-01-23T13:00:00-03:00,Visit,Cancelled
4,182396,1,2022-01-23T10:00:00-03:00,2022-01-23T12:00:00-03:00,Visit,Cancelled


In [56]:
# Revisamos el typo de los datos para ver si necesitamos hacer transformaciones y ver donde puede haber valores nulos
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425 entries, 0 to 424
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   scheduled_id  425 non-null    int64 
 1   property_id   425 non-null    int64 
 2   begin_date    425 non-null    object
 3   end_date      425 non-null    object
 4   type_visit    425 non-null    object
 5   status        425 non-null    object
dtypes: int64(2), object(4)
memory usage: 20.0+ KB


In [65]:
# Nos interesa saber los tipos de status y los tipos de visitas que existen
print(visits.status.value_counts())
print(visits.type_visit.value_counts())
print(visits.type_visit.nunique())

Cancelled    286
Done         139
Name: status, dtype: int64
Visit    425
Name: type_visit, dtype: int64
1


In [58]:
# Con el método de decribe entendemos si hay outliers y dónde se concentran los datos
visits.describe()

,scheduled_id,property_id
count,425.000000,425.000000
mean,188094.894118,40.087059
std,13820.402675,22.943521
min,160130.000000,1.000000
25%,176671.000000,21.000000
50%,190273.000000,38.000000
75%,200307.000000,61.000000
max,208926.000000,80.000000


In [55]:
# Hay dos columnas de fechas, hay que entender la primera y la última
print("Fecha de la primer visita: ",visits.begin_date.min())
print("Fecha de la ultima visita: ",visits.begin_date.max())

print("Primer fecha de término: ",visits.end_date.min())
print("Última fecha de término ", visits.end_date.max())

Fecha de la primer visita:  2022-01-01T16:00:00-03:00
Fecha de la ultima visita:  2022-02-28T21:00:00-03:00
Primer fecha de término:  2022-01-01T18:00:00-03:00
Última fecha de término  2022-02-28T23:00:00-03:00


<b>Notas</b>
<li>Solo existe un tipo de visita</li>
<li>Hay 425 visitas de las cuales 286 fueron canceladas y 139 completadas</li>
<li>La primer visista fue en 1-Enero y la última el 28 de Febrero de 2022</li>
<li>La relación de esta tabla con la de propiedades es de uno a varios, es decir, por una propiedad existen varias visitas.

## Preguntas del desafío 

#### 1. ¿Cuántas visitas se realizaron en total?

In [100]:
# Con realizadas se puede referir a las que fueron completadas, pero daremos una respuesta más completa
freq = visits.status.value_counts()

# Respuesta
print("Respuesta: ")
print("Total de visitas: ", sum(freq))
print("Visitas canceladas: ", freq.Cancelled)
print("Visitas realizadas: ", freq.Done)

Respuesta: 
Total de visitas:  425
Visitas canceladas:  286
Visitas realizadas:  139


#### 2. ¿Cuál es el promedio de propiedades por propietario?

Para esta pregunta nos sirvió el análisis de usuarios dónde saber el hecho de que la relación sea de 1 usuario por propiedad no daría una respuesta evidente. Sin embargo hagamos el proceso completo.

In [95]:
# Contamos el número de propiedades desde la tabla de propiedades:
total = properties.property_id.nunique()

# Propiedades por usuario:
properties_per_user = list(users.property_id.value_counts())

# Promedio de propiedades por usuario:
print("Promedio de propiedades por propietario: ", sum(properties_per_user)/total)

Promedio de propiedades por propietario:  1.0


##### Respuesta:
El promedio de propiedades por cada propietario es de:     __1__

#### 3. ¿Cuál era la temperatura promedio de todas las visitas que realizó en la propiedad del propietario con ID 2?

Para responder esta pregunta necesitaremos aprovechar la conexión entre las 3 tablas. Para esto con el usuario del id = 2 podemos filtrar la información de la propiedad que le pertenece a ese usuario (property_id, latitude, longitude). Con esta información posteriormente en la tabla de visitas filtraremos con el id de la propiedad para acceder a las fechas de las visitas. Con las fechas, latitud, longitud y la KEY de nuestra cuenta podremos acceder a la información de temperatures que queremos de la api de https://weather.visualcrossing.com/.

In [250]:
# Localizamos el id de la propiedad del usuario con id = 2
property_id_2 = int(users.loc[users["user_id"] == 2].property_id)

# Localizamos las visitas en la propiedad con el id que encontramos
visits_id_2 = visits.loc[visits["property_id"] == property_id_2]

# Extraemos la latitud y la longitud de la propiedad para posteriormente conectarnos con la api
latitude = float(properties.loc[properties["property_id"] == property_id_2].latitude)
longitude = float(properties.loc[properties["property_id"] == property_id_2].longitude)

print("ID de la propiedad del usuario con id = 2: ", property_id_2)
print("latitude", latitude)
print("longitude", longitude)

# Revisamos el total de visitas
print(visits_id_2.shape)

visits_id_2.head(10)

ID de la propiedad del usuario con id = 2:  34
latitude 4.6183634
longitude -74.0735
(7, 6)


,scheduled_id,property_id,begin_date,end_date,type_visit,status
177,186033,34,2022-01-29T12:00:00-03:00,2022-01-29T14:00:00-03:00,Visit,Cancelled
178,187080,34,2022-01-28T11:00:00-03:00,2022-01-29T01:00:00-03:00,Visit,Done
179,186857,34,2022-01-28T11:30:00-03:00,2022-01-28T14:00:00-03:00,Visit,Done
180,180698,34,2022-01-22T20:00:00-03:00,2022-01-22T22:00:00-03:00,Visit,Cancelled
181,177506,34,2022-01-23T13:00:00-03:00,2022-01-23T15:00:00-03:00,Visit,Cancelled
182,178172,34,2022-01-21T18:00:00-03:00,2022-01-21T20:00:00-03:00,Visit,Cancelled
183,177505,34,2022-01-23T09:00:00-03:00,2022-01-23T11:00:00-03:00,Visit,Cancelled


Ahora que tenemos la información necesaria, vamos a crear una función que nos permita extraer las temperaturas de la api en un rango específico de fechas y probarla con la primer visita.

In [245]:
def get_temperatures(latitude:str, longitude:str, api_key:str, start_date:str, end_date:str) -> list:
    """
        Return a list of temperatures corresponding to the latitude and location in a specific range of dates.
        
        params:
            latitude:   latitude coordinate.
            longitude:  longitude coordinate.
            api_key:    key extracted from //vissualcrossing.com// account .
            start_date: begin date from range of dates to get temperatures with format 'YYYY-MM-DD'
            end_date:   end date from range of dates to get temperatures 'YYYY-MM-DD'.
        return:
            temperatures: temperatures of the days in the range selected
    """
    
    # Get request from the API: https://www.visualcrossing.com/weather-api
    response = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{latitude},{longitude}/{start_date}/{end_date}?key={api_key}&include=days")
    
    # Temperaures for every day in range selected
    temperatures = [days["temp"] for days in response.json()["days"]]
    
    return temperatures

# Extraer las fechas de la primer visita
begin_date = visits_id_2.iloc[1,:].begin_date.split('T')[0]
end_date = visits_id_2.iloc[1,:].end_date.split('T')[0]

# API_KEY de la cuenta de visualcrossing
API_KEY = "KFLMT278G4E5LGCP6WEWFTXNY"

# Get temperatures for fist visit
temperatures = get_temperatures(latitude=latitude,
                               longitude=longitude,
                               api_key=API_KEY,
                               start_date=begin_date,
                               end_date=end_date)
    
# Get the mean of the temperatures    
mean_temperature =  np.mean(temperatures)

print("Temperaturas en el primer rango de dias: ", temperatures)
print("Temperatura promedio: ", mean_temperature)

Temperaturas en el primer rango de dias:  [58.3, 56.1]
Temperatura promedio:  57.2


<li>Hemos visto que la api de retorna las temperaturas del rango de días, por lo que fue necesario obtener su promedio.</li>
<li>Para responder la pregunta vamos a aprovechar la función que creamos y correrla en todo el rango de fechas de las visitas a la propiedad.</li>

In [251]:
# Temperatura promedio para cada dia:
temperatures_avg = []

for dates in zip(visits_id_2.begin_date, visits_id_2.end_date):
    
    # Extraer las fechas de todas las visitas
    begin_date = dates[0].split('T')[0]
    end_date = dates[1].split('T')[0]
    
    # Get temperatures for every visit
    temperatures_avg.append(np.mean((get_temperatures(latitude=latitude, longitude=longitude, api_key=API_KEY, start_date=begin_date, end_date=end_date))))
                        
print("Temperaturas promedio para cada rango de fechas: ", temperatures_avg)                        

Temperaturas promedio para cada rango de fechas:  [56.1, 57.2, 58.3, 58.1, 59.3, 57.7, 59.3]


Con las temperaturas de todas las visitas ahora si podemos acceder al resultado promediandolas

In [256]:
# Promedio de temperaturas
temperatures_average = np.mean(temperatures)

print("Promedio de temperaturas: ", temperatures_average)

Promedio de temperaturas:  57.2


##### Respuesta:
La temperatura promedio de los días en que se realizaron las visitas en la propiedad del usuario con id = 2 es:     __57.2__

#### 4. ¿Cuál es la temperatura promedio de las visitas para los días con lluvia?

Buscaremos un campo en la respuesta de la api que nos permita definir si llovió en el día especificado

In [287]:
# Ejemplo de las condicionts de la respuesta para el primer día de las visitas de la pregunta pasada
begin_date = visits_id_2.iloc[0, :].begin_date.split('T')[0]
end_date = visits_id_2.iloc[0, :].end_date.split('T')[0]

# Get request from the API: https://www.visualcrossing.com/weather-api
response = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{latitude},{longitude}/{begin_date}/{end_date}?key={API_KEY}&include=days")
date_json = response.json()
date_json["days"][0]

{'datetime': '2022-01-29',
 'datetimeEpoch': 1643432400,
 'tempmax': 68.6,
 'tempmin': 42.8,
 'temp': 56.1,
 'feelslikemax': 68.6,
 'feelslikemin': 42.8,
 'feelslike': 56.0,
 'dew': 46.6,
 'humidity': 73.9,
 'precip': 0.01,
 'precipprob': 100.0,
 'precipcover': 4.17,
 'preciptype': ['rain'],
 'snow': 0.0,
 'snowdepth': 0.0,
 'windgust': 9.4,
 'windspeed': 15.8,
 'winddir': 260.2,
 'pressure': 1025.0,
 'cloudcover': 49.2,
 'visibility': 7.2,
 'solarradiation': 318.2,
 'solarenergy': 27.8,
 'uvindex': 10.0,
 'severerisk': 10.0,
 'sunrise': '06:11:56',
 'sunriseEpoch': 1643454716,
 'sunset': '18:07:04',
 'sunsetEpoch': 1643497624,
 'moonphase': 0.95,
 'conditions': 'Rain, Partially cloudy',
 'description': 'Partly cloudy throughout the day with late afternoon rain.',
 'icon': 'rain',
 'stations': ['80222099999', '80222399999', 'SKBO', '80234099999', 'SKVV'],
 'source': 'obs'}

En la respuesta de la api de vissuacrossing existe un campo que se llama "conditions". A partir de este campo podemos de terminar si ese día llovió buscando que incluya la palabra "Rain".

Para poder hacer la búsqueda en la API por cada visita necesitaremos los datos de latitude y longitude de la propiedad visitada, por lo que haremos un merge entre ambas tablas basándonos en el id de la propiedad.

In [3]:
# hacemos un merge entre las propiedades y las visitas
visits_properties_df = pd.merge(left=visits, 
                                 right=properties, 
                                 how='inner', 
                                 on=['property_id', 'property_id'])

print(visits_properties_df.shape)
visits_properties_df.head()

(425, 20)


,scheduled_id,property_id,begin_date,end_date,type_visit,status,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
0,169548,1,2022-01-13T10:00:00-03:00,2022-01-13T12:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
1,184763,1,2022-01-26T18:00:00-03:00,2022-01-26T20:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
2,186092,1,2022-01-28T12:00:00-03:00,2022-01-28T14:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
3,182497,1,2022-01-23T11:00:00-03:00,2022-01-23T13:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
4,182396,1,2022-01-23T10:00:00-03:00,2022-01-23T12:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia


Crearemos una función que nos permita conectarnos a la api, y nos devuelva toda la información que podamos necesitar, desde la respuesta de la api hasta las temperaturas de cada visita.

In [49]:
def get_avg_temperature_of_rainy_days(latitude:str, longitude:str, api_key:str, start_date:str, end_date:str, scheduled_id:int) -> str:
    """
        Return the average temperature of the day if it was a rainy day.
        
        params:
            latitude:     latitude coordinate.
            longitude:    longitude coordinate.
            api_key:      key extracted from //vissualcrossing.com// account .
            start_date:   begin date from range of dates to get temperatures with format 'YYYY-MM-DD'
            end_date:     end date from range of dates to get temperatures 'YYYY-MM-DD'.
            scheduled_id: id of scheduled visit from visits data.
        return:
            temperature_avg: temperature of the day if it was rainy.
    """
    
    returned_values = {}
    temperatures = []
    response = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{latitude},{longitude}/{start_date}/{end_date}?key={api_key}&include=days")
    
    if response.status_code == 200:
        
        # Guardar temperaturas de los días en los que llovió
        for day in response.json()["days"]:
            if "Rain" in day["conditions"]:
                temperatures.append(day["temp"])
        
        # Obtener el promedio de los valores
        returned_values["scheduled"] = scheduled_id 
        returned_values["temperatures"] = temperatures        
        returned_values["temperature_avg"] = np.mean(temperatures)        
        
    # En caso de devolver un error, guardar el id de la visita y el estatus del error            
    else:
        print(scheduled_id, " ", response.status_code)
        returned_values["temperature_avg"] = None
        returned_values["scheduled_id"] = scheduled_id
        returned_values["status_code"] = response.status_code
    
    return returned_values, response
    

In [98]:
# Ejemplo de las condicionts de la respuesta para el primer día de las visitas de la pregunta pasada
API_KEY = "3NHKCVGUTYRMZJDJLE76WV5U6"

# Extraemos la información de la visita
begin_date = visits_properties_df.loc[visits_properties_df["scheduled_id"] == 169548].begin_date.values[0].split('T')[0]
end_date = visits_properties_df.loc[visits_properties_df["scheduled_id"] == 169548].end_date.values[0].split('T')[0]
longitude = str(visits_properties_df.loc[visits_properties_df["scheduled_id"] == 169548].longitude.values[0])
latitude = str(visits_properties_df.loc[visits_properties_df["scheduled_id"] == 169548].latitude.values[0])

t_avg, response = get_avg_temperature_of_rainy_days(latitude, longitude, API_KEY, begin_date, end_date, scheduled_id=169548)
t_avg

{'scheduled': 169548, 'temperatures': [55.1], 'temperature_avg': 55.1}

In [22]:
# API_KEY de la cuenta de visualcrossing
API_KEY = "3NHKCVGUTYRMZJDJLE76WV5U6"

# Haremos el request para cada visita y lo guardamos en una lista
temperatures = []

for i in range(0 , len(visits_properties_df)):
    start_date = visits_properties_df.iloc[i].begin_date.split('T')[0]
    end_date = visits_properties_df.iloc[i].end_date.split('T')[0]
    longitude = visits_properties_df.iloc[i].longitude
    latitude = visits_properties_df.iloc[i].latitude
    visit_id = visits_properties_df.iloc[i].scheduled_id
    
    # Llamamos la función de la API que ya entrega el promedio si llovió en esos días
    temperatures.append(get_avg_temperature_of_rainy_days(latitude=latitude, longitude=longitude, api_key=API_KEY, start_date=start_date, end_date=end_date, scheduled_id=visit_id))
    

In [59]:
# Transformamos los datos de temperatures en dataframe para realizar operaciones con facilidad
df_temperatures = pd.DataFrame(temperatures)
rainy_days = len(df) - df['temperature_avg'].isnull().sum()

print('Total de dias lluviosos: ', rainy_days)

Total de dias lluviosos:  403


In [74]:
# Eliminamos los valores nulos y sacamos el promedio
df_temperatures_notnull = df_temperatures.dropna(subset='temperature_avg')
avg = np.mean(df_temperatures_notnull.temperature_avg)
print("Temperatura promedio: ", avg)

Temperatura promedio:  57.34590570719603


##### Respuesta:
La temperatura promedio de los días de visita en los que llovio es de:     __57.3__

#### 5. ¿Cuál es la temperatura promedio para las visitas realizadas en la localidad de Suba?

Para responder esta pregunta vamos a aprovechar la tabla generada en el ejercicio pasado, donde la información de las visitas y de las propiedades ya está unidas.

In [76]:
visits_properties_df.head()

,scheduled_id,property_id,begin_date,end_date,type_visit,status,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
0,169548,1,2022-01-13T10:00:00-03:00,2022-01-13T12:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
1,184763,1,2022-01-26T18:00:00-03:00,2022-01-26T20:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
2,186092,1,2022-01-28T12:00:00-03:00,2022-01-28T14:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
3,182497,1,2022-01-23T11:00:00-03:00,2022-01-23T13:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia
4,182396,1,2022-01-23T10:00:00-03:00,2022-01-23T12:00:00-03:00,Visit,Cancelled,departamento,Rental,1,1,1,3.0,0,False,4.870956,-74.05804,Chí­A,Cundinamarca,Región De Cundinamarca,Colombia


In [79]:
# Filtraremos la tabla solo para la localidad de Suba
suba_data = visits_properties_df.loc[visits_properties_df['localidad'] == 'Suba']
print(suba_data.shape)
suba_data.head(2)

(86, 20)


,scheduled_id,property_id,begin_date,end_date,type_visit,status,type_house,business_type,bedrooms,bathrooms,parking_lots,services,balcony,pool,latitude,longitude,localidad,city,region,country
21,164788,4,2022-01-03T10:00:00-03:00,2022-01-03T12:00:00-03:00,Visit,Cancelled,departamento,Rental & Sale,3,2,1,0.0,0,False,4.721716,-74.057976,Suba,Bogotá,Región De Cundinamarca,Colombia
39,166464,9,2022-01-07T11:00:00-03:00,2022-01-07T13:00:00-03:00,Visit,Cancelled,departamento,Rental,3,1,0,3.0,0,False,4.733718,-74.107970,Suba,Bogotá,Región De Cundinamarca,Colombia


In [84]:
# A pesar de ser la misma localidad buscaremos si hay diferentes coordenadas
print('Latitudes', suba_data.latitude.nunique())
print('Longitudes', suba_data.longitude.nunique())

Latitudes 14
Longitudes 14


A partir de esto sabemos que a la api la alimentaremos con distintos datos por cada visita

In [86]:
# Haremos el request para cada visita y lo guardamos en una lista
temperatures_avg = []
API_KEY = "3NHKCVGUTYRMZJDJLE76WV5U6"

for i in range(0 , len(suba_data)):
    start_date = suba_data.iloc[i].begin_date.split('T')[0]
    end_date = suba_data.iloc[i].end_date.split('T')[0]
    longitude = suba_data.iloc[i].longitude
    latitude = suba_data.iloc[i].latitude
    
    # Reutilizamos la funcion de las temperatures del tercer ejercicio
    temperatures_avg.append(np.mean((get_temperatures(latitude=latitude, longitude=longitude, api_key=API_KEY, start_date=start_date, end_date=end_date))))


In [92]:
# Obtenemos el promedio de las temperatures
mean_temperature = np.mean(temperatures_avg)
print('Temperatura promedio para visitas en Suba: ', mean_temperature)

Temperatura promedio para visitas en Suba:  57.6017441860465


##### Respuesta:
La temperatura promedio para las visitas realizadas en la localidad de Suba es de:     __57.6__

## Conclusiones

Me pareció un gran ejercicio para trabajar con datos locales y coordinarlos con fuentes de externas, en este caso de la api de las condiciones climatológicas. Con más tiempo se podrían optimizar:
<li>Ciertos métodos para extraer datos de la API y no lidiar con el hecho de que las llamadas a esta nos pueden limitar.</li>
<li>También se pueden generalizar aún más las funciones para que se puedan responder estás preguntas más con el sólo hecho de introducir ciertos parámetros como localidad, id del propiertario, o de condiciones climatológicas.</li>
<li>Analitica para encontrar patrones e insights en la información</li>

	Gracias por darme la oportunidad de participar en el proceso. Sería un honor trabajar con el equipo de datos!